<a href="https://colab.research.google.com/github/fmathews11/Football_Field_Goals/blob/main/Predicting_Field_Goals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, re
from pandas.api.types import is_string_dtype
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
pd.set_option('display.max_columns', None)
from google.colab import output
output.clear()
df = pd.read_csv('https://raw.githubusercontent.com/fmathews11/Football_Field_Goals/main/historical_field_goal_data.csv')
#print(*((idx,i) for idx,i in enumerate(df.columns)),sep = '\n')
df = df.drop([df.columns[i] for i in [0,11,12,31,33]],axis =1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9286 entries, 0 to 9285
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   week                 9286 non-null   int64  
 1   away                 9286 non-null   object 
 2   home                 9286 non-null   object 
 3   stadium              9286 non-null   object 
 4   temperature          7034 non-null   float64
 5   humidity             6787 non-null   float64
 6   wind_speed           6899 non-null   float64
 7   wind_direction       6881 non-null   object 
 8   condition            9250 non-null   object 
 9   surface              9286 non-null   object 
 10  away_score           9286 non-null   int64  
 11  home_score           9286 non-null   int64  
 12  off                  9286 non-null   object 
 13  def                  9286 non-null   object 
 14  quarter              9286 non-null   int64  
 15  minutes              9286 non-null   i

In [11]:
columns_to_encode = [i for i in pd.Series({col:df[col].nunique() for col in df if is_string_dtype(df[col])}).index]
le_dict = {key:LabelEncoder() for key in columns_to_encode}
for col in le_dict.keys():
  print(f"Label Encoding {col}")
  df[col] = le_dict[col].fit_transform(df[col])
df

Label Encoding away
Label Encoding home
Label Encoding stadium
Label Encoding wind_direction
Label Encoding condition
Label Encoding surface
Label Encoding off
Label Encoding def


,week,away,home,stadium,temperature,humidity,wind_speed,wind_direction,condition,surface,away_score,home_score,off,def,quarter,minutes,seconds,offense_score,defense_score,timeouts_offense,timeouts_defense,down,yards_to_go,yards_from_own_goal,points,penalty,fumble,block,kicker_id,distance,made
0,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,23,8,2,9,14,0,0,3,3,4,4,96,3,0,0,0,2886,22,1
1,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,8,23,3,0,45,14,10,2,3,4,15,85,3,0,0,0,4452,33,1
2,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,2,5,36,14,7,1,3,4,4,83,3,0,0,0,2820,35,1
3,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,13,5,2,0,6,14,24,1,1,2,10,81,0,0,0,0,2888,37,0
4,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,3,10,7,31,14,3,3,4,6,92,3,0,0,0,2820,26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,20,3,15,2,40.0,70.0,7.0,6,4,5,24,38,3,15,4,3,19,21,38,3,3,4,28,67,3,0,0,0,54578,51,1
9282,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,1,5,14,0,0,3,3,4,11,69,3,0,0,0,53628,49,1
9283,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,2,1,4,3,14,3,2,4,6,86,3,0,0,0,53628,34,1
9284,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,3,11,31,6,21,3,3,4,7,66,3,0,0,0,53628,52,1


In [12]:
df.isna().sum()[lambda x: x >1]

temperature    2252
humidity       2499
wind_speed     2387
dtype: int64

In [13]:
#Filtering by stadiums with a minimum of 50 kicks recorded
df = df[df.stadium.isin(df.groupby('stadium').size().sort_values()[lambda x: x > 50].index.tolist())]

#Imputing the wind direction with the most common value by stadium
df['wind_direction_mode'] = df.groupby('stadium').wind_direction.transform(lambda x: x.value_counts().index[0])

df.wind_direction = df.wind_direction.fillna(df.wind_direction_mode)
df.drop('wind_direction_mode',axis = 1,inplace= True)
df

,week,away,home,stadium,temperature,humidity,wind_speed,wind_direction,condition,surface,away_score,home_score,off,def,quarter,minutes,seconds,offense_score,defense_score,timeouts_offense,timeouts_defense,down,yards_to_go,yards_from_own_goal,points,penalty,fumble,block,kicker_id,distance,made
0,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,23,8,2,9,14,0,0,3,3,4,4,96,3,0,0,0,2886,22,1
1,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,8,23,3,0,45,14,10,2,3,4,15,85,3,0,0,0,4452,33,1
2,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,2,5,36,14,7,1,3,4,4,83,3,0,0,0,2820,35,1
3,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,13,5,2,0,6,14,24,1,1,2,10,81,0,0,0,0,2888,37,0
4,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,3,10,7,31,14,3,3,4,6,92,3,0,0,0,2820,26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,20,3,15,2,40.0,70.0,7.0,6,4,5,24,38,3,15,4,3,19,21,38,3,3,4,28,67,3,0,0,0,54578,51,1
9282,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,1,5,14,0,0,3,3,4,11,69,3,0,0,0,53628,49,1
9283,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,2,1,4,3,14,3,2,4,6,86,3,0,0,0,53628,34,1
9284,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,3,11,31,6,21,3,3,4,7,66,3,0,0,0,53628,52,1


In [14]:
def assign_month(x):
  if x.week in [1,2,3]: return 9
  elif x.week in [4,5,6,7,8]: return 10
  elif x.week in [9,10,11,12]: return 11
  elif x.week in [13,14,15,16]: return 12
  elif x.week in [17,18,19,20]:return 1
  else: return 2
df['month'] = df.apply(lambda x: assign_month(x),axis = 1)
df

,week,away,home,stadium,temperature,humidity,wind_speed,wind_direction,condition,surface,away_score,home_score,off,def,quarter,minutes,seconds,offense_score,defense_score,timeouts_offense,timeouts_defense,down,yards_to_go,yards_from_own_goal,points,penalty,fumble,block,kicker_id,distance,made,month
0,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,23,8,2,9,14,0,0,3,3,4,4,96,3,0,0,0,2886,22,1,9
1,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,8,23,3,0,45,14,10,2,3,4,15,85,3,0,0,0,4452,33,1,9
2,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,2,5,36,14,7,1,3,4,4,83,3,0,0,0,2820,35,1,9
3,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,13,5,2,0,6,14,24,1,1,2,10,81,0,0,0,0,2888,37,0,9
4,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,3,10,7,31,14,3,3,4,6,92,3,0,0,0,2820,26,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,20,3,15,2,40.0,70.0,7.0,6,4,5,24,38,3,15,4,3,19,21,38,3,3,4,28,67,3,0,0,0,54578,51,1,1
9282,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,1,5,14,0,0,3,3,4,11,69,3,0,0,0,53628,49,1,2
9283,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,2,1,4,3,14,3,2,4,6,86,3,0,0,0,53628,34,1,2
9284,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,3,11,31,6,21,3,3,4,7,66,3,0,0,0,53628,52,1,2


In [15]:
imp = IterativeImputer(initial_strategy='most_frequent')
df_imputed = pd.DataFrame(imp.fit_transform(df),columns = df.columns)
df_imputed

,week,away,home,stadium,temperature,humidity,wind_speed,wind_direction,condition,surface,away_score,home_score,off,def,quarter,minutes,seconds,offense_score,defense_score,timeouts_offense,timeouts_defense,down,yards_to_go,yards_from_own_goal,points,penalty,fumble,block,kicker_id,distance,made,month
0,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,23,8,2,9,14,0,0,3,3,4,4,96,3,0,0,0,2886,22,1,9
1,1,8,23,32,77.0,63.0,5.0,16,8,4,24,17,8,23,3,0,45,14,10,2,3,4,15,85,3,0,0,0,4452,33,1,9
2,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,2,5,36,14,7,1,3,4,4,83,3,0,0,0,2820,35,1,9
3,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,13,5,2,0,6,14,24,1,1,2,10,81,0,0,0,0,2888,37,0,9
4,1,13,5,45,67.0,66.0,13.0,5,18,5,21,41,5,13,3,10,7,31,14,3,3,4,6,92,3,0,0,0,2820,26,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007,20,3,15,2,40.0,70.0,7.0,6,4,5,24,38,3,15,4,3,19,21,38,3,3,4,28,67,3,0,0,0,54578,51,1,1
9008,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,1,5,14,0,0,3,3,4,11,69,3,0,0,0,53628,49,1,2
9009,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,2,1,4,3,14,3,2,4,6,86,3,0,0,0,53628,34,1,2
9010,21,15,32,41,63.0,78.0,9.0,9,1,5,9,31,15,32,3,11,31,6,21,3,3,4,7,66,3,0,0,0,53628,52,1,2


In [ ]:
for col in ['temperature', 'humidity', 'wind_speed', 'wind_direction', 'condition']:
  plt.figure(figsize = (10,8))
  sns.histplot(data = df,
               x = col)
  plt.title(f"Histogram of {col.title()} before imputation")
  plt.show();
  plt.figure(figsize = (10,8))
  sns.histplot(data = df,
               x = col)
  plt.title(f"Histogram of {col.title()} after imputation")
  plt.show();

In [17]:
from xgboost import XGBClassifier
clf = XGBClassifier()

In [22]:
print(str(clf)[:-2])

XGBClassifier


In [26]:
def train_and_evaluate_model(X,y,model):

  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 23,stratify = y)
  print(str(clf)[:-2])
  model.fit(X_train,y_train)
  print("Training Accuracy")
  print(cross_val_score(model,X_train,y_train))

In [29]:
train_and_evaluate_model(df_imputed.drop("made",axis =1),df_imputed.made,clf)

XGBClassifier


ValueError: ignored

In [28]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9012 entries, 0 to 9011
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   week                 9012 non-null   Int64  
 1   away                 9012 non-null   Int64  
 2   home                 9012 non-null   Int64  
 3   stadium              9012 non-null   Int64  
 4   temperature          9012 non-null   Float64
 5   humidity             9012 non-null   Float64
 6   wind_speed           9012 non-null   Float64
 7   wind_direction       9012 non-null   Int64  
 8   condition            9012 non-null   Int64  
 9   surface              9012 non-null   Int64  
 10  away_score           9012 non-null   Int64  
 11  home_score           9012 non-null   Int64  
 12  off                  9012 non-null   Int64  
 13  def                  9012 non-null   Int64  
 14  quarter              9012 non-null   Int64  
 15  minutes              9012 non-null   I